# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Quality-and-Index-Hopping" data-toc-modified-id="Quality-and-Index-Hopping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Quality and Index Hopping</a></div><div class="lev2 toc-item"><a href="#Quality-Distributions" data-toc-modified-id="Quality-Distributions-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Quality Distributions</a></div><div class="lev3 toc-item"><a href="#Plotting-script" data-toc-modified-id="Plotting-script-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Plotting script</a></div><div class="lev2 toc-item"><a href="#Index-Hopping" data-toc-modified-id="Index-Hopping-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Index Hopping</a></div><div class="lev3 toc-item"><a href="#Index-hopping-script" data-toc-modified-id="Index-hopping-script-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Index hopping script</a></div><div class="lev3 toc-item"><a href="#Results" data-toc-modified-id="Results-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Results</a></div><div class="lev2 toc-item"><a href="#Questions" data-toc-modified-id="Questions-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Questions</a></div><div class="lev3 toc-item"><a href="#Lines-in-files" data-toc-modified-id="Lines-in-files-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Lines in files</a></div>

# Quality and Index Hopping
**Kohl Kinning**

Generate per base call distribution of quality scores for read1, read2, index1, and index2. Generate a per nucleotide distribution. Next, average the Quality scores for each read (for each of the four files) and plot frequency of the Quality Scores

## Quality Distributions

### Plotting script

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import argparse


def get_arguments():
    parser = argparse.ArgumentParser(prog='qscore_plot', description= 'Outputs a distribution of quality scores as a function of the base position of sequencing reads.')

    parser.add_argument('-f', '--infile', help='fastq filepath', required=True, type=argparse.FileType('rt', encoding='UTF-8 '))

    return parser.parse_args()

def convert_phred(letter):
    score = ord(letter) - 33    # phred+33
    return score


args = get_arguments()
file = args.infile.name

all_scores = np.zeros(101)
mean_read_scores = Counter()


with open(file, 'r') as fh:
    count = 0
    NR = 1
    for line in fh:
        line = str(line).strip('\n')
        count = 2
        if NR % 4 == 0:  # quality line
            ntd_pos = 0
            read_score = 0
            for ntd in line:
                all_scores[ntd_pos] = ((int(convert_phred(ntd)) + all_scores[ntd_pos]) / (count))

                read_score += int(convert_phred(ntd))
                read_avg = int(read_score // len(line))  # floor div
                ntd_pos += 1
            count += 1

            mean_read_scores[read_avg] += 1
        NR += 1 

# quality score distribution of reads
xdata = np.arange(0, len(all_scores[all_scores > 0]), 1)
plt.figure()
plt.bar(xdata, all_scores[all_scores > 0], width = 0.5)

print(all_scores[all_scores > 0])

plt.title(str(file) + '\n' + 'Mean Quality Score by Base Position')
plt.xlabel('Base Position')
plt.ylabel('Mean Quality Score')
plt.savefig(file + '_distribReads.png')



# quality score distribution across all reads
x = []
y = []
for key in mean_read_scores:
    x.append(key)
    y.append(mean_read_scores[key])


plt.figure()
plt.bar(x, y, width = 0.5)
plt.title(str(file) + '\n' + 'Frequency of Average Quality Score Across All Reads')
plt.xlabel('Avg score per read')
plt.ylabel('Frequency')
plt.savefig(file + '_distribAll.png')

## Index Hopping

Write a program to de-multiplex the samples and document index swapping and number of reads retained per sample.

### Index hopping script

In [ ]:
#!/usr/bin/env python3.6

import argparse
import itertools
from collections import Counter


def complement(seq):
    complement = {'C': 'G', 'T': 'A', 'A': 'T',  'G': 'C', 'N': 'N'}
    ntds = list(seq)
    ntds = [complement[ntd] for ntd in ntds]
    return ''.join(ntds)


def reverse_complement(s):
        return complement(s[::-1])


def get_arguments():
    parser = argparse.ArgumentParser(prog='index_hopping', description= 'Demultiplexing of paired end sequences by index paring')
    parser.add_argument('-1', '--R1', help='R1 file path', required=True, type=argparse.FileType('rt', encoding='UTF-8 '))
    parser.add_argument('-2', '--R2', help='R2 file path', required=True, type=argparse.FileType('rt', encoding='UTF-8 '))
    parser.add_argument('-3', '--R3', help='R3 file path', required=True, type=argparse.FileType('rt', encoding='UTF-8 '))
    parser.add_argument('-4', '--R4', help='R4 file path', required=True, type=argparse.FileType('rt', encoding='UTF-8 '))
    parser.add_argument('-c', '--minScore', help='Quality score cutoff', required=True, type=int)
    parser.add_argument('-i', '--ind', help='Index file path', required=True, type=argparse.FileType('rt', encoding='UTF-8 '))

    return parser.parse_args()


args = get_arguments()

min_score = args.qcutoff

R1 = args.R1.name
R2 = args.R2.name
R3 = args.R3.name
R4 = args.R4.name
index = args.ind.name


index_dict = {}

with open(index, 'r') as ind:
    for line in ind:
        line = line.strip('\n').split('\t')
        index_dict[line[3]] = line[4]


all_indices = []

for key in index_dict:
    all_indices.append(index_dict[key])


all_pairs_list = list(itertools.combinations_with_replacement(all_indices, 2))

all_pairs_dict = {}

for pair in all_pairs_list:
    all_pairs_dict[pair] = 0

undetermined_dict = Counter()

with open(R1 ,'r') as r1, open(R2, 'r') as r2, open(R3, 'r') as r3, open(R4, 'r') as r4:
    NL = 0
    
    for line in zip(r1, r2, r3, r4):
        line = [l.strip() for l in line]
        if NL % 4 == 0:
            head = line
        if NL % 4 == 1:
            seq = line
        if NL % 4 == 2:
            plus = line
        if NL % 4 == 3:
            qline = line
            for j in qline[1:2]:
                qlist = []
                qtotal = 0
                for qual in j:
                    qtotal += ord(qual) - 33
                    qavg = qtotal/len(j)
                    if qavg >= min_score:
                        qlist.append(qual)
                        pair = seq[1], reverse_complement(seq[2])
                        if pair in all_pairs_dict:
                            all_pairs_dict[pair] += 1
                        else:
                            undetermined_dict[pair] += 1
        NL += 1


match = open('matched.tsv', 'w')
match.write('Matched Index Pair\tCounts\n')

swap = open('swapped.tsv', 'w')
swap.write('Swapped Index Pair\tCounts\n')

und = open('undetermined.tsv', 'w')
und.write('Undetermined Index Pair\tCounts\n')


swapped = 0
for pair in all_pairs_dict:
    if pair[0] == pair[1]:
        match.write(str(pair[0]) + '_' + str(pair[1]) + '\t' + str(all_pairs_dict[pair]) + '\n')
    else:
        swapped += 1
        swap.write(str(pair[0]) + '_' + str(pair[1]) + '\t' + str(all_pairs_dict[pair]) + '\n')

for pair in undetermined_dict:
    und.write((str(pair[0]) + '_' + str(pair[1]) + '\t' + str(undetermined_dict[pair]) + '\n'))


matched = 0
for pair in all_pairs_dict:
    matched += all_pairs_dict[pair]

undetermined = 0
for com in undetermined_dict:
    undetermined += undetermined_dict[com]

stats = open('metrics.tsv', 'w')
stats.write('Filtered' + '\t' + 'Matched' + '\t' + 'Swapped' + '\t' + 'Undetermined' + '\n')
stats.write(str(matched+swapped+undetermined) + '\t' + str(matched) + '\t' + str(swapped) + '\t' + str(undetermined)+'\n')

### Results

#### Matches

In [13]:
import csv

matched_list = []

with open('matched.tsv','r') as matched:
    matched = csv.reader(matched, delimiter='\t')
    for row in matched:
        matched_list.append(row)
        
matched_list[:10]

[['Matched Index Pair', 'Counts'],
 ['index sequence_index sequence', '0'],
 ['GTAGCGTA_GTAGCGTA', '15170760'],
 ['CGATCGAT_CGATCGAT', '10424338'],
 ['GATCAAGG_GATCAAGG', '12168683'],
 ['AACAGCGA_AACAGCGA', '16933442'],
 ['TAGCCATG_TAGCCATG', '19843352'],
 ['CGGTAATC_CGGTAATC', '9246665'],
 ['CTCTGGAT_CTCTGGAT', '65953935'],
 ['TACCGGAT_TACCGGAT', '146346803']]

#### Swapped indexes

In [12]:
import csv

swapped_list = []

with open('swapped.tsv','r') as swapped:
    swapped = csv.reader(swapped, delimiter='\t')
    for row in swapped:
        swapped_list.append(row)
        
swapped_list[:10]

[['Swapped Index Pair', 'Counts'],
 ['TAGCGTA_CGATCGAT', '307'],
 ['GTAGCGTA_GATCAAGG', '385'],
 ['GTAGCGTA_AACAGCGA', '513'],
 ['GTAGCGTA_TAGCCATG', '458'],
 ['GTAGCGTA_CGGTAATC', '227'],
 ['GTAGCGTA_CTCTGGAT', '1344'],
 ['GTAGCGTA_TACCGGAT', '2869'],
 ['GTAGCGTA_CTAGCTCA', '1293'],
 ['GTAGCGTA_CACTTCAC', '173']]

#### Undetermined

In [15]:
import csv

und_list = []

with open('undetermined.tsv','r') as und:
    und = csv.reader(und, delimiter='\t')
    for row in und:
        und_list.append(row)
        
und_list[:10]

[['Undetermined Index Pair', 'Counts'],
 ['NCTTCGAC_TCTTCGAN', '339360'],
 ['NACAGCGA_AACAGCGN', '71775'],
 ['NTCCTAAG_GTCCTAAN', '69054'],
 ['NATGGCAC_TATGGCAN', '88482'],
 ['NACCGGAT_TACCGGAN', '617101'],
 ['NTCTGGAT_CTCTGGAN', '276960'],
 ['NCGAGAGT_TCGAGAGN', '85296'],
 ['NGGATAGC_AGGATAGN', '69022'],
 ['NTCATGCG_ATCATGCN', '78633']]

## Questions

***What is a good quality score cutoff for index reads and pairs to utilize for sample identification and downstream analysis, respectively?***

I chose a cutoff of 30. My plots showed the vast majority of the quality scores as being above this limit.

***How many indexes have Undetermined (N) base calls? (Utilize your command line tool knowledge. Summit the command you used. CHALLENGE: use a one line command)***

```
$ awk "NR%4==2" /projects/bgmp/2017_sequencing/1294_S1_L008_R2_001.fastq | grep "N" | wc -l
$ 3976613

$ awk "NR%4==2" /projects/bgmp/2017_sequencing/1294_S1_L008_R3_001.fastq | grep "N" | wc -l
$ 3328051
```

***What do the averaged Quality Scores across the reads tell you? Interpret your data specifically.***

The averaged quality scores are (to me) surprisingly high. I've read in the literature that 20 is a common cutoff value. I believe we had a high quality sequencing run.

***How many reads are retained for each expected index pair? What is the percentage?***

232,228,144 reads remained average being filtered by the cutoff value. Of these, 226,715,602 had appropriately matched indexes. There were 363,246,735 in total. ~62.4% remain 

***How many reads are indicative of index swapping?***

330,975 reads were indicative of index swapping.

***Create a distribution of swapped indexes. What does this histogram tell you/what is your interpretation of this data?***

See `/plots/swappedDist.pdf`

Though the plot is impossible to label legibly with index pairs as labels, we can still make use of the histogram. The important thing that it shows us is that the distribution of swapped indexes is not normal. There are some indexes that are found to be swapped at a much higher frequency than others.